In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
from sympy.matrices import Matrix, eye
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction

from scipy.linalg import lu, inv

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
# make sure to run #BIOMD 18
# #112 is very tricky to detect errors (no errors?!)
# Biomodels 383 can be used to test reduction 
simple = load_file_from_curated_data(383)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

PGA_prod_Vc: RuBP + CO2 + 2.00 NADPH -> 2.00 PGA
PGA_prod_Vo: RuBP + CO2 + 2.00 NADPH -> 1.50 PGA
PGA_cons: PGA -> RuBP
NADPH_prod: NADP -> NADPH


In [3]:
m = GAMES_PP(simple)

In [4]:
m.getStoichiometryMatrix(simple.reactions, simple.molecules)

,PGA_prod_Vc,PGA_prod_Vo,PGA_cons,NADPH_prod
RuBP,-1.0,-1.0,1.0,0.0
CO2,-1.0,-1.0,0.0,0.0
NADPH,-2.0,-2.0,0.0,1.0
PGA,2.0,1.5,-1.0,0.0
NADP,0.0,0.0,0.0,-1.0


In [5]:
PGA_CONS = "PGA_cons"
PGA_PROD_VC = "PGA_prod_Vc"
reaction1 = m.simple.getReaction(PGA_CONS)
reaction2 = m.simple.getReaction(PGA_PROD_VC)
mat = m.getStoichiometryMatrix(m.reactions, m.molecules)
print(m.convertReactionToSOMReaction(reaction1))
print(m.convertReactionToSOMReaction(reaction2))

PGA_cons: {PGA} -> {RuBP}
PGA_prod_Vc: {RuBP} + 2.00 {NADPH} + {CO2} -> 2.00 {PGA}


In [6]:
m.analyze()

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [PGA_prod_Vo: 0.50 {PGA, RuBP} -> ]
Type III error:  []
Type I-SOM error:  set()


True

In [17]:
m.identifier

'{CO2}->{PGA, RuBP}\n{NADP, NADPH}->{PGA, RuBP}\n'

In [7]:
m.type_three_errors

[]

In [8]:
p, l, u = lu(m.som_stoichiometry_matrix.T)

In [9]:
inv(l)

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.33333333,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.2       ,  0.6       ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        , 

In [119]:
mat_df = m.som_stoichiometry_matrix
mat_t = mat_df.T
idx_mat_t = mat_t.index
cols_mat_t = mat_t.columns
#################################
if mat_t.shape[0] > mat_t.shape[1]:
  diff = mat_t.shape[0] - mat_t.shape[1]
  for i in range(diff):
    mat_t["_" + str(i)] = np.zeros(mat_t.shape[0])
#################################
# LU decomposition
perm, lower, upper = lu(mat_t)
# # the following is applicable to transposed stoichiometry matrix
# # 'perm' is an orthogonal matrix, meaning perm.T == inv(perm)
# ## permuted_m = inv(perm).dot(mat_t)
perm_inverse = perm.T
permuted_m = (perm_inverse).dot(mat_t)
pivot_index = [list(k).index(1) for k in perm_inverse]
new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
# # we save as; perm_df * lower_operation * mat_t = rref_df
if diff:
  perm_df = pd.DataFrame(permuted_m,
      index=new_idx_mat_t,
      columns=mat_t.columns).drop(columns = mat_t.columns[-diff:]).T
  rref_df = pd.DataFrame(upper,
      index=new_idx_mat_t,
      columns=mat_t.columns).drop(columns = mat_t.columns[-diff:]).T
else:
  perm_df = pd.DataFrame(permuted_m,
      index=new_idx_mat_t,
      columns=mat_t.columns).T
  rref_df = pd.DataFrame(upper,
      index=new_idx_mat_t,
      columns=mat_t.columns).T
lower_inverse = pd.DataFrame(inv(lower),
     columns=new_idx_mat_t)
# self.permuted_matrix = perm_df
# self.lower_inverse = lower_inverse
# self.rref_df = rref_df

In [120]:
lower_inverse

,v11,v3,v7,v9,v5,v1,v12
0,1.0,0.0,0.0,0.0,0.0,0.0,-0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,-0.0
2,0.0,-1.0,1.0,0.0,0.0,0.0,-0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,-0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,-0.0
5,0.0,-1.0,0.0,0.0,0.0,1.0,-0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [112]:
perm_df

,v11,v3,v7,v9,v5,v1,v12
{Pr},1.0,0.00,0.00,0.00,0.00,0.00,-1.0
{Ca_cyt},1.0,1.00,1.00,-1.00,-1.00,1.00,-1.0
{CaPr},-1.0,0.00,0.00,0.00,0.00,0.00,1.0
{CaER},0.0,-0.25,0.00,0.00,0.25,-0.25,0.0
{CaM},0.0,0.00,-0.25,0.25,0.00,0.00,0.0


,{Pr},{Ca_cyt},{CaPr},{CaER},{CaM},_0,_1
v11,1.0,1.0,-1.0,0.00,0.00,0.0,0.0
v3,0.0,1.0,0.0,-0.25,0.00,0.0,0.0
v7,0.0,0.0,0.0,0.25,-0.25,0.0,0.0
v9,0.0,0.0,0.0,-0.25,0.25,0.0,0.0
v5,0.0,0.0,0.0,0.00,0.00,0.0,0.0
v1,0.0,0.0,0.0,0.00,0.00,0.0,0.0
v12,0.0,0.0,0.0,0.00,0.00,0.0,0.0


In [99]:
mat_t.columns[-diff:]

Index(['_0', '_1'], dtype='object')

In [105]:
pd.DataFrame(upper,
    index=new_idx_mat_t,
    columns=mat_t.columns).drop(columns = mat_t.columns[-diff:]).T

,v11,v3,v7,v9,v5,v1,v12
{Pr},1.0,0.00,0.00,0.00,0.0,0.0,0.0
{Ca_cyt},1.0,1.00,0.00,0.00,0.0,0.0,0.0
{CaPr},-1.0,0.00,0.00,0.00,0.0,0.0,0.0
{CaER},0.0,-0.25,0.25,-0.25,0.0,0.0,0.0
{CaM},0.0,0.00,-0.25,0.25,0.0,0.0,0.0


In [103]:
k.drop(columns = mat_t.columns[-diff:])

,{Pr},{Ca_cyt},{CaPr},{CaER},{CaM}
v11,1.0,1.0,-1.0,0.00,0.00
v3,0.0,1.0,0.0,-0.25,0.00
v7,0.0,0.0,0.0,0.25,-0.25
v9,0.0,0.0,0.0,-0.25,0.25
v5,0.0,0.0,0.0,0.00,0.00
v1,0.0,0.0,0.0,0.00,0.00
v12,0.0,0.0,0.0,0.00,0.00


In [88]:
rref_df

,v11,v3,v7,v9,v5,v1,v12
{Pr},1.0,0.00,0.00,0.00,0.0,0.0,0.0
{Ca_cyt},1.0,1.00,0.00,0.00,0.0,0.0,0.0
{CaPr},-1.0,0.00,0.00,0.00,0.0,0.0,0.0
{CaER},0.0,-0.25,0.25,-0.25,0.0,0.0,0.0
{CaM},0.0,0.00,-0.25,0.25,0.0,0.0,0.0
_0,0.0,0.00,0.00,0.00,0.0,0.0,0.0
_1,0.0,0.00,0.00,0.00,0.0,0.0,0.0


In [78]:
rref_df.columns[-diff:]

Index(['v1', 'v12'], dtype='object')

In [77]:
-diff

-2

In [89]:
rref_df.drop(rref_df.T.columns[-diff:])

,v11,v3,v7,v9,v5,v1,v12
{Pr},1.0,0.00,0.00,0.00,0.0,0.0,0.0
{Ca_cyt},1.0,1.00,0.00,0.00,0.0,0.0,0.0
{CaPr},-1.0,0.00,0.00,0.00,0.0,0.0,0.0
{CaER},0.0,-0.25,0.25,-0.25,0.0,0.0,0.0
{CaM},0.0,0.00,-0.25,0.25,0.0,0.0,0.0


In [85]:
list(range(-2, 0))

[-2, -1]

In [86]:
rref_df.drop([0, 1])

ValueError: labels [0 1] not contained in axis

In [33]:
print(idx_mat_t)
print(new_idx_mat_t)

Index(['v1', 'v3', 'v7', 'v11', 'v5', 'v9', 'v12'], dtype='object')
['v11', 'v3', 'v7', 'v9', 'v5', 'v1', 'v12']


In [34]:
perm_inverse

array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [10]:
m.som_stoichiometry_matrix.T

,{Pr},{Ca_cyt},{CaPr},{CaER},{CaM}
v1,0.0,1.0,0.0,-0.25,0.00
v3,0.0,1.0,0.0,-0.25,0.00
v7,0.0,1.0,0.0,0.00,-0.25
v11,1.0,1.0,-1.0,0.00,0.00
v5,0.0,-1.0,0.0,0.25,0.00
v9,0.0,-1.0,0.0,0.00,0.25
v12,-1.0,-1.0,1.0,0.00,0.00


In [11]:
m.lower_inverse

In [13]:
sm = m.som_stoichiometry_matrix.T
print(sm.shape[0])
print(sm.shape[1])
sm

7
5


,{Pr},{Ca_cyt},{CaPr},{CaER},{CaM}
v1,0.0,1.0,0.0,-0.25,0.00
v3,0.0,1.0,0.0,-0.25,0.00
v7,0.0,1.0,0.0,0.00,-0.25
v11,1.0,1.0,-1.0,0.00,0.00
v5,0.0,-1.0,0.0,0.25,0.00
v9,0.0,-1.0,0.0,0.00,0.25
v12,-1.0,-1.0,1.0,0.00,0.00


In [14]:
if sm.shape[0] > sm.shape[1]:
  diff = sm.shape[0] - sm.shape[1]
  for i in range(diff):
    sm["_" + str(i)] = np.zeros(sm.shape[0])
sm

,{Pr},{Ca_cyt},{CaPr},{CaER},{CaM},_0,_1
v1,0.0,1.0,0.0,-0.25,0.00,0.0,0.0
v3,0.0,1.0,0.0,-0.25,0.00,0.0,0.0
v7,0.0,1.0,0.0,0.00,-0.25,0.0,0.0
v11,1.0,1.0,-1.0,0.00,0.00,0.0,0.0
v5,0.0,-1.0,0.0,0.25,0.00,0.0,0.0
v9,0.0,-1.0,0.0,0.00,0.25,0.0,0.0
v12,-1.0,-1.0,1.0,0.00,0.00,0.0,0.0


In [23]:
m.decomposeMatrix(sm.T)

,v11,v3,v7,v9,v5,v1,v12
{Pr},1.0,0.00,0.00,0.00,0.0,0.0,0.0
{Ca_cyt},1.0,1.00,0.00,0.00,0.0,0.0,0.0
{CaPr},-1.0,0.00,0.00,0.00,0.0,0.0,0.0
{CaER},0.0,-0.25,0.25,-0.25,0.0,0.0,0.0
{CaM},0.0,0.00,-0.25,0.25,0.0,0.0,0.0
_0,0.0,0.00,0.00,0.00,0.0,0.0,0.0
_1,0.0,0.00,0.00,0.00,0.0,0.0,0.0


In [19]:
m.lower_inverse

,{Ca_cyt},{Pr},{CaER},{CaPr},{CaM},_0,_1
0,1.00,0.0,0.0,0.00,0.0,0.0,0.0
1,0.00,1.0,0.0,0.00,0.0,0.0,0.0
2,-0.25,0.0,1.0,0.00,0.0,0.0,0.0
3,0.00,0.0,0.0,1.00,0.0,0.0,0.0
4,0.00,0.0,-1.0,-0.25,1.0,0.0,0.0
5,0.00,0.0,0.0,-0.00,0.0,1.0,0.0
6,0.00,0.0,0.0,-0.00,0.0,0.0,1.0


In [22]:
inv(m.lower_inverse)

array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ],
       [ 0.25,  0.  ,  1.  ,  0.  ,  0.  ,  0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  , -0.  ],
       [ 0.25,  0.  ,  1.  ,  0.25,  1.  ,  0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ]])

In [24]:
m.reactions_lu

[reaction_6: R_smad_P_smad4_nuc -> smad4_nuc + R_smad_P_nuc; k6d * R_smad_P_smad4_nuc - k6a * smad4_nuc * R_smad_P_nuc,
 reaction_7: R_smad_P_nuc -> R_smad_nuc + Pi; Vmax7 * R_smad_P_nuc / (K7 + R_smad_P_nuc),
 reaction_2: R_smad_P_cyt + smad4_cyt -> R_smad_P_smad4_cyt; k2a * R_smad_P_cyt * smad4_cyt - k2d * R_smad_P_smad4_cyt]

In [25]:
m.som_stoichiometry_matrix

,reaction_6,reaction_7,reaction_2
{R_smad_P_nuc},1.0,-1.0,0.0
{Pi},0.0,1.0,0.0
"{R_smad_P_cyt, R_smad_cyt, R_smad_nuc}",0.0,1.0,-1.0
"{smad4_cyt, smad4_nuc}",1.0,0.0,-1.0
"{R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}",-1.0,0.0,1.0


In [29]:
m.decomposeMatrix(m.som_stoichiometry_matrix).T

,{R_smad_P_nuc},{Pi},"{R_smad_P_cyt, R_smad_cyt, R_smad_nuc}","{smad4_cyt, smad4_nuc}","{R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}"
reaction_6,1.0,0.0,0.0,1.0,-1.0
reaction_7,0.0,1.0,1.0,1.0,-1.0
reaction_2,0.0,0.0,-1.0,-1.0,1.0


In [30]:
m.reduced_som_reactions

[reaction_6: {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc} -> {R_smad_P_nuc} + {smad4_cyt, smad4_nuc},
 reaction_7: {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc} -> {Pi} + {R_smad_P_cyt, R_smad_cyt, R_smad_nuc} + {smad4_cyt, smad4_nuc},
 reaction_2: {R_smad_P_cyt, R_smad_cyt, R_smad_nuc} + {smad4_cyt, smad4_nuc} -> {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}]

In [31]:
m.processUnequalSOMReaction(m.reduced_som_reactions[0])
m.edges

OutEdgeView([({R_smad_P_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}), ({Pi}, {R_smad_P_nuc}), ({Pi}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}), ({R_smad_P_cyt, R_smad_cyt, R_smad_nuc}, {R_smad_P_nuc}), ({R_smad_P_cyt, R_smad_cyt, R_smad_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}), ({smad4_cyt, smad4_nuc}, {R_smad_P_smad4_cyt, R_smad_P_smad4_nuc})])

In [32]:
reaction = m.reduced_som_reactions[0]
category = reaction.category
if (category!=cn.REACTION_n_1) or (category!=cn.REACTION_1_n):
  print("noo")
# else:
#   if category == cn.REACTION_n_1:
#     destination = [reaction.products[0].molecule]
#     source = [reactant.molecule for reactant in reaction.reactants]
#   elif category == cn.REACTION_1_n:
#     destination = [reaction.reactants[0].molecule]
#     source = [product.molecule for product in reaction.products]
#   arcs = itertools.product(source, destination)
#   for arc in arcs:
#     if arc[0] == arc[1]:
#       self.type_one_som_errors = self.type_one_som_errors.add(reaction.label)
#     else:
#       self.addArc(arc[0], arc[1], reactin.label)

noo


In [33]:
m.lower_inverse.dot(m.permuted_matrix.T)

,{R_smad_P_nuc},{Pi},"{R_smad_P_cyt, R_smad_cyt, R_smad_nuc}","{smad4_cyt, smad4_nuc}","{R_smad_P_smad4_cyt, R_smad_P_smad4_nuc}"
0,1.0,0.0,0.0,1.0,-1.0
1,-2.0,1.0,1.0,-1.0,1.0
2,0.0,0.0,-1.0,-1.0,1.0


In [34]:
p, l, u = lu(m.stoichiometry_matrix.T)
p

array([[0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [37]:
p.dot(p.T)

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [14]:
u

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [15]:
l.dot(u)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.]])

In [16]:
# this is the matrix for operation (without transpose)
reduce_operation = inv(l).dot(inv(p))
reduce_operation

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [15]:
print(m.analyze())
m.rref_errors

False


[]

In [6]:
if [] or [] or []:
  print("none")

In [10]:
rref_df = m.decomposeMatrix(m.stoichiometry_matrix)
reactions = m.convertMatrixToReactions(simple, rref_df)

In [11]:
reactions

[ReactionSummary(label='reaction6', reactants=[XI *  1.00], products=[X *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction5', reactants=[M *  1.00], products=[MI *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction4', reactants=[], products=[], category='reaction_redundant'),
 ReactionSummary(label='reaction7', reactants=[], products=[], category='reaction_redundant')]

In [12]:
k = m.processUniUniReaction(reactions[0])

In [13]:
type(reactions[0].reactants[0].molecule)

SBMLLint.common.molecule.Molecule

In [14]:
str(reactions[0].label)

'reaction6'

In [15]:
m2 = MESGraph(simple)
m2.analyze()

Model analyzed...
No error detected.


''

In [16]:
s = StoichiometryMatrix(simple)
s.isConsistent()

True

In [17]:
list(m2.nodes)[1].reactions

{reaction4: MI -> M; cell * MI * V1 * pow(K1 + MI, -1)}

In [2]:
import networkx as nx

NameError: name 'm' is not defined